In [1]:
import os
from sklearn.model_selection import train_test_split
import shutil
import pandas as pd
import random
random.seed(7)

In [2]:
base_dir = '../'
chainsaw_dir = base_dir+'ESC-50-master/wav_chainsaw_5sec/'
no_chainsaw_dir = base_dir+'ESC-50-master/split_wav/'

result_base_dir = base_dir+'data/'

In [3]:
merge_list = []

chainsaw_filenames = [chainsaw_dir+filename for filename in os.listdir(chainsaw_dir)
                         if filename.endswith('.wav')]
chainsaw_filenames = [filename for filename in chainsaw_filenames
                         if os.path.getsize(filename) == 441078]
chainsaw_filenames = random.sample(chainsaw_filenames, 1500)


x_train_chainsaw, x_test_chainsaw \
    = train_test_split(chainsaw_filenames, test_size = 0.3, random_state = 7)
x_train_chainsaw, x_val_chainsaw \
    = train_test_split(x_train_chainsaw, test_size = 0.3, random_state = 7)
    
merge_list += [(os.path.basename(filename), 1, 'train') for filename in x_train_chainsaw]
merge_list += [(os.path.basename(filename), 1, 'val') for filename in x_val_chainsaw]
merge_list += [(os.path.basename(filename), 1, 'test') for filename in x_test_chainsaw]

no_chainsaw_dirs = [no_chainsaw_dir+dirname for dirname in os.listdir(no_chainsaw_dir)
                       if os.path.isdir(no_chainsaw_dir+dirname)]
for dirname in no_chainsaw_dirs:
    no_chainsaw_filenames = [dirname+'/'+filename for filename in os.listdir(dirname)
                             if filename.endswith('.wav')]
    
    x_train_no_chainsaw, x_test_no_chainsaw \
        = train_test_split(no_chainsaw_filenames, test_size = 0.33, random_state = 7)
    x_train_no_chainsaw, x_val_no_chainsaw \
        = train_test_split(x_train_no_chainsaw, test_size = 0.33, random_state = 7)
        
    merge_list += [(os.path.basename(filename), 0, 'train') for filename in x_train_no_chainsaw]
    merge_list += [(os.path.basename(filename), 0, 'val') for filename in x_val_no_chainsaw]
    merge_list += [(os.path.basename(filename), 0, 'test') for filename in x_test_no_chainsaw]

In [4]:
merge_list[:10]

[('392_26.17.wav', 1, 'train'),
 ('450_50.1.wav', 1, 'train'),
 ('8_29.05.wav', 1, 'train'),
 ('597_263.75.wav', 1, 'train'),
 ('1548_27.76.wav', 1, 'train'),
 ('1147_182.92.wav', 1, 'train'),
 ('1321_40.58.wav', 1, 'train'),
 ('1598_6.38.wav', 1, 'train'),
 ('1528_40.34.wav', 1, 'train'),
 ('379_14.32.wav', 1, 'train')]

In [5]:
df = pd.DataFrame({
                    'filename':[filename for filename, _, _ in merge_list],
                    'label':[label for _, label, _ in merge_list],
                    'type':[data_type for _, _, data_type in merge_list]
                  })
df = df.sample(frac=1).reset_index(drop=True)
df.head

<bound method NDFrame.head of                filename  label   type
0         1001_21.6.wav      1  train
1         1055_48.7.wav      1    val
2     3-103051-A-19.wav      0   test
3     4-165791-B-45.wav      0  train
4        1224_17.94.wav      1  train
5        1189_18.15.wav      1   test
6     1-155858-F-25.wav      0  train
7     4-128659-A-33.wav      0  train
8     4-166661-A-10.wav      0  train
9     4-125072-B-19.wav      0   test
10        284_42.37.wav      1   test
11       1133_28.28.wav      1   test
12    2-188822-B-40.wav      0   test
13        952_71.66.wav      1    val
14        345_18.41.wav      1   test
15        897_23.67.wav      1    val
16    2-262579-A-45.wav      0    val
17        919_10.12.wav      1  train
18     1-67152-A-17.wav      0  train
19       1266_51.58.wav      1  train
20    1-100038-A-14.wav      0    val
21     1-24524-B-19.wav      0  train
22        169_86.91.wav      1   test
23        978_80.08.wav      1    val
24       1179_19.14.

In [6]:
df.to_csv('1200_data_annotations.csv', index=False)

In [7]:
df = pd.read_csv('1200_data_annotations.csv')

data_filename_list = df['filename'].tolist()

chainsaw_filenames = [chainsaw_dir+filename for filename in os.listdir(chainsaw_dir)
                         if filename.endswith('.wav')]
os.makedirs(result_base_dir, exist_ok=True)
for filename in chainsaw_filenames:
    if os.path.basename(filename) in data_filename_list:
        shutil.copy(filename, result_base_dir)

no_chainsaw_dirs = [no_chainsaw_dir+dirname for dirname in os.listdir(no_chainsaw_dir)
                       if os.path.isdir(no_chainsaw_dir+dirname)]
for dirname in no_chainsaw_dirs:
    no_chainsaw_filenames = [dirname+'/'+filename for filename in os.listdir(dirname)
                             if filename.endswith('.wav')]
    for filename in no_chainsaw_filenames:
        if os.path.basename(filename) in data_filename_list:
            shutil.copy(filename, result_base_dir)
    
for idx, filename, label, data_type in df.itertuples():
    result_dir = result_base_dir+data_type+'/'
    os.makedirs(result_dir, exist_ok=True)
    shutil.move(result_base_dir+filename, result_dir)